In [14]:
from scipy.sparse import csr_matrix
import numpy as np
raw_file = open("ml-1m.tsv", "r")
lines = raw_file.readlines()

In [46]:
edge = [[], []]
label = []
for line in lines:
    line = line.strip("/n")
    line = line.split("\t")
    fr, to, sign = line
    edge[0].append(int(fr))
    edge[1].append(int(to))
    label.append(sign)


In [47]:
num_user = max(edge[0])+1
num_user

6040

In [48]:
num_item = max(edge[1])+1
num_item

3952

In [49]:
num_nodes = num_item+num_user

In [82]:
import torch
A = torch.sparse_coo_tensor(torch.tensor(edge), torch.tensor([1]*len(edge[0])), (num_nodes, num_nodes))
A.shape

torch.Size([9992, 9992])

In [86]:
row_sum = torch.sum(A, dim=1)

In [91]:
d_inv_row = torch.pow(row_sum, -0.5).flatten()
d_inv_matrix = torch.diag(d_inv_row.to_dense()).shape

torch.Size([9992, 9992])

In [51]:
from scipy.sparse import spdiags
def row_normalize(A):
    '''
    Perform row-normalization of the given matrix

    inputs
        A : crs_matrix
            (n x n) input matrix where n is # of nodes
    outputs
        nA : crs_matrix
             (n x n) row-normalized matrix
    '''
    n = A.shape[0]

    # do row-wise sum where d is out-degree for each node
    d = A.sum(axis=1)
    d = np.asarray(d).flatten()

    # handle 0 entries in d
    d = np.maximum(d, np.ones(n))
    invd = 1.0 / d

    invD = spdiags(invd, 0, n, n)

    # compute row normalized adjacency matrix by nA = invD * A
    nA = invD.dot(A)

    return nA

In [52]:
nA = row_normalize(A)
nAT = nA.T

In [53]:
nAT.data

array([0.01886792, 0.01886792, 0.01886792, ..., 0.00293255, 0.00293255,
       0.00293255])

In [56]:
import torch
nAT = nAT.tocoo()
row = torch.tensor(nAT.row, dtype=torch.long)
col = torch.tensor(nAT.col, dtype=torch.long)
values = torch.tensor(nAT.data, dtype=torch.float)
indices = torch.stack([row, col], dim=0)
torch_sparse_tensor = torch.sparse.FloatTensor(indices, values, torch.Size(nAT.shape))

In [79]:
alpha = 0.8
x0 = torch.randn(num_nodes, num_nodes).to("cuda:0")
I = torch.eye(num_nodes).to("cuda:0")
nAt = torch_sparse_tensor.to("cuda:0")
x = x0.to("cuda:0")
for i in range(3):
    x = alpha * nAt.matmul(x) + (1-alpha) * I

In [80]:
x

tensor([[0.2000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2000]],
       device='cuda:0')

In [73]:
alpha = 0.8
x0 = torch.randn(num_nodes, num_nodes).abs().to("cuda:0")
I = torch.eye(num_nodes).to("cuda:0")
nAt = torch_sparse_tensor.to("cuda:0")
x = x0.to("cuda:0")
for i in range(30):
    x = alpha * nAt.matmul(x) + (1-alpha) * I